# Aircraft Maintenance Data Cleaning & Analysis

In [1]:
#Import required libraries

import pandas as pd
import numpy as np

In [ ]:
# Load the uncleaned dataset

df = pd.read_excel("../data/aircraft_maintenance_uncleaned.xlsx")

# Confirm dataset loaded

df.head()

# A. Duplicates & Missing Values

In [6]:
# 1. Count duplicate rows in the dataset

df.duplicated().sum()

In [10]:
# 2. Count duplicates based on columns

df.duplicated(subset = ["Maintenance_ID","Aircraft_ID"]).sum()

In [12]:
# 3. Remove duplicate rows

df = df.drop_duplicates()

In [14]:
# 4. Count missing values in each column

df.isnull().sum()

In [16]:
# 5. Calculate missing value percentage per column

(df.isnull().sum() / len(df) * 100).round(2)

In [17]:
# 6. Drop rows where operationally critical columns are missing

df = df.dropna(subset=['Scheduled_Date', 'Maintenance_Category'])

In [18]:
# 7. Fill missing numeric columns using column mean

numeric_columns = [
    'Budgeted_Cost', 'Actual_Cost', 'Labor_Hours',
    'Parts_Cost', 'Downtime_Hours',
    'Technician_Count', 'Cost_Variance',
    'Cost_Variance_Percent'
]

for col in numeric_columns:
    df[col] = df[col].fillna(df[col].mean())

In [19]:
# 8. Fill missing categorical columns using mode

categorical_columns = [
    'Aircraft_Type', 'Maintenance_Type',
    'Maintenance_Category', 'Vendor',
    'Priority', 'Status', 'Location'
]

for col in categorical_columns:
    df[col] = df[col].fillna(df[col].mode()[0])

In [20]:
# 9. Conditional filling based on business logic

condition = (df['Status'] == 'Completed') & (df['Actual_Cost'].isnull())
df.loc[condition, 'Actual_Cost'] = df.loc[condition, 'Budgeted_Cost']

In [24]:
# 10. Validate duplicate removal and count rows still containing any missing values

df.duplicated().sum(), df.isnull().any(axis=1).sum()

# B. Invalid / Outlier Values

In [48]:
# 1. Validate that all cost columns are positive

(df[['Budgeted_Cost', 'Actual_Cost', 'Parts_Cost']] <= 0).any(axis=1).sum()

np.int64(0)

In [49]:
# 2. Correct invalid cost values using median replacement

df[['Budgeted_Cost', 'Actual_Cost', 'Parts_Cost']] = df[
    ['Budgeted_Cost', 'Actual_Cost', 'Parts_Cost']
].apply(lambda x: x.where(x > 0, x.median()))

In [50]:
# 3. Detect labor hours exceeding downtime hours

(df['Labor_Hours'] > df['Downtime_Hours']).sum()

np.int64(1590)

In [51]:
# 4. Align labor hours to downtime hours where invalid

df.loc[df['Labor_Hours'] > df['Downtime_Hours'], 'Labor_Hours'] = df['Downtime_Hours']

In [53]:
# 5. Identify rows with invalid downtime hours

invalid_downtime = df[
    (df['Downtime_Hours'] < 0) |
    (df['Downtime_Hours'] > 400)
]

In [54]:
# 6. Normalize downtime hours to monthly upper limit

df.loc[df['Downtime_Hours'] > 24 * 30, 'Downtime_Hours'] = 24 * 30

In [55]:
# 7. Detect incorrect cost variance values

(df['Cost_Variance'] != df['Actual_Cost'] - df['Budgeted_Cost']).sum()

np.int64(0)

In [47]:
# 8. Recalculate cost variance using actual and budgeted cost

df['Cost_Variance'] = df['Actual_Cost'] - df['Budgeted_Cost']

In [58]:
# 9. Detect technician count inconsistent with labor hours

(df['Technician_Count'] > df['Labor_Hours']).sum()

In [57]:
# 10. Adjust technician count based on labor hours

df.loc[df['Technician_Count'] > df['Labor_Hours'], 'Technician_Count'] = df['Labor_Hours']

# C. Conditional Fetch / Filtering

In [60]:
# 1. Fetch records where maintenance status is Completed

completed_maintenance = df[df['Status'] == 'Completed']

In [62]:
# 2. Fetch records where maintenance is Delayed

delayed_maintenance = df[df['Status'] == 'Delayed']

In [63]:
# 3. Fetch high-priority maintenance records

high_priority = df[df['Priority'] == 'Critical']

In [65]:
# 4. Fetch maintenance records with downtime greater than 100 hours

high_downtime = df[df['Downtime_Hours'] > 100]

In [67]:
# 5. Fetch maintenance records for a specific location

london_maintenance = df[df['Location'] == 'London']

In [68]:
# 6. Fetch records where actual cost exceeded budgeted cost

over_budget = df[df['Actual_Cost'] > df['Budgeted_Cost']]

In [69]:
# 7. Fetch emergency maintenance records

emergency_maintenance = df[df['Maintenance_Type'] == 'Emergency']

In [70]:
# 8. Fetch completed maintenance records with high cost

completed_high_cost = df[
    (df['Status'] == 'Completed') &
    (df['Actual_Cost'] > df['Actual_Cost'].median())
]

In [71]:
# 9. Fetch maintenance records within a specific year

maintenance_2023 = df[
    pd.to_datetime(df['Scheduled_Date']).dt.year == 2023
]

In [72]:
# 10. Fetch maintenance records with both high downtime and high labor hours

complex_maintenance = df[
    (df['Downtime_Hours'] > 100) &
    (df['Labor_Hours'] > 150)
]

# E. Calculated Columns / Transformation

In [73]:
# 1. Convert date columns to datetime format

df['Scheduled_Date'] = pd.to_datetime(df['Scheduled_Date'])
df['Actual_Start_Date'] = pd.to_datetime(df['Actual_Start_Date'])
df['Completion_Date'] = pd.to_datetime(df['Completion_Date'])

In [74]:
# 2. Fill missing Actual_Start_Date using Scheduled_Date

df.loc[df['Actual_Start_Date'].isnull(), 'Actual_Start_Date'] = df['Scheduled_Date']

In [75]:
# 3. Fill missing Completion_Date using Actual_Start_Date plus 1 day

df.loc[df['Completion_Date'].isnull(), 'Completion_Date'] = (
    df['Actual_Start_Date'] + pd.Timedelta(days=1)
)

In [76]:
# 4. Create Maintenance_Duration column in days

df['Maintenance_Duration'] = (
    df['Completion_Date'] - df['Actual_Start_Date']
).dt.days

In [77]:
# 5. Create Cost_Variance_Percent column

df['Cost_Variance_Percent'] = (
    (df['Actual_Cost'] - df['Budgeted_Cost']) / df['Budgeted_Cost']
) * 100

In [78]:
# 6. Create Delay_Flag column based on schedule vs actual start

df['Delay_Flag'] = np.where(
    df['Actual_Start_Date'] > df['Scheduled_Date'],
    'Yes',
    'No'
)

In [79]:
# 7. Create High_Cost_Flag based on median actual cost

df['High_Cost_Flag'] = np.where(
    df['Actual_Cost'] > df['Actual_Cost'].median(),
    'Yes',
    'No'
)

In [80]:
# 8. Extract Maintenance_Year from Scheduled_Date

df['Maintenance_Year'] = df['Scheduled_Date'].dt.year

In [81]:
# 9. Extract Maintenance_Month from Scheduled_Date

df['Maintenance_Month'] = df['Scheduled_Date'].dt.month

In [82]:
# 10. Normalize text columns to consistent format

text_columns = ['Aircraft_Type', 'Maintenance_Type', 'Vendor', 'Status', 'Location']
df[text_columns] = df[text_columns].apply(lambda x: x.str.strip())

# F. Sorting / Ranking / Aggregation

In [84]:
# 1. Sort maintenance records by highest actual cost

sorted_by_cost = df.sort_values(by='Actual_Cost', ascending=False)

In [85]:
# 2. Sort maintenance records by longest downtime

sorted_by_downtime = df.sort_values(by='Downtime_Hours', ascending=False)

In [86]:
# 3. Rank aircraft by total maintenance cost

aircraft_cost_rank = (
    df.groupby('Aircraft_ID')['Actual_Cost']
      .sum()
      .sort_values(ascending=False)
)

In [87]:
# 4. Rank locations by total downtime hours

location_downtime_rank = (
    df.groupby('Location')['Downtime_Hours']
      .sum()
      .sort_values(ascending=False)
)

In [89]:
# 5. Aggregate total maintenance cost by location

location_total_cost = (
    df.groupby('Location')['Actual_Cost']
      .sum()
)

In [91]:
# 6. Aggregate average maintenance cost by aircraft type

aircraft_type_avg_cost = (
    df.groupby('Aircraft_Type')['Actual_Cost']
      .mean()
)

In [92]:
# 7. Count maintenance records by status

status_count = df['Status'].value_counts()

In [93]:
# 8. Identify top 5 most expensive maintenance records

top_5_expensive = df.sort_values(by='Actual_Cost', ascending=False).head(5)

In [94]:
# 9. Identify bottom 5 maintenance records by downtime

bottom_5_fastest = df.sort_values(by='Downtime_Hours', ascending=True).head(5)

In [96]:
# 10. Create a summary table by location

location_summary = (
    df.groupby('Location')
      .agg(
          Total_Maintenance=('Maintenance_ID', 'count'),
          Total_Cost=('Actual_Cost', 'sum'),
          Avg_Downtime=('Downtime_Hours', 'mean')
      )
)

# G. Advanced Cleaning & Validation

In [98]:
# 1. Validate logical order of maintenance dates

invalid_date_order = df[
    (df['Scheduled_Date'] > df['Actual_Start_Date']) |
    (df['Actual_Start_Date'] > df['Completion_Date'])
]

In [99]:
# 2. Correct invalid date order by aligning dates sequentially

df.loc[df['Actual_Start_Date'] < df['Scheduled_Date'], 'Actual_Start_Date'] = df['Scheduled_Date']
df.loc[df['Completion_Date'] < df['Actual_Start_Date'], 'Completion_Date'] = df['Actual_Start_Date']

In [100]:
# 3. Recalculate maintenance duration after date correction

df['Maintenance_Duration'] = (
    df['Completion_Date'] - df['Actual_Start_Date']
).dt.days

In [101]:
# 4. Ensure maintenance duration is non-negative

df.loc[df['Maintenance_Duration'] < 0, 'Maintenance_Duration'] = 0

In [102]:
# 5. Validate numeric columns are of correct data type

numeric_columns = [
    'Budgeted_Cost', 'Actual_Cost', 'Labor_Hours',
    'Parts_Cost', 'Downtime_Hours',
    'Technician_Count', 'Cost_Variance',
    'Cost_Variance_Percent', 'Maintenance_Duration'
]

df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

In [104]:
# 6. Validate no negative values exist in numeric columns

(df[numeric_columns] < 0).any(axis=1).sum()

In [105]:
# 7. Validate technician count does not exceed labor hours

invalid_tech_labor = df[df['Technician_Count'] > df['Labor_Hours']]

In [106]:
# 8. Correct technician count where it exceeds labor hours

df.loc[df['Technician_Count'] > df['Labor_Hours'], 'Technician_Count'] = df['Labor_Hours']

In [108]:
# 9. Validate consistency between cost variance and cost variance percent

variance_mismatch = df[
    df['Cost_Variance_Percent'].round(2) !=
    ((df['Actual_Cost'] - df['Budgeted_Cost']) / df['Budgeted_Cost'] * 100).round(2)
]

In [110]:
# 10. Validate rows that pass all core data quality rules

valid_rows = df[
    (df['Budgeted_Cost'] > 0) &
    (df['Actual_Cost'] > 0) &
    (df['Labor_Hours'] >= 1) &
    (df['Technician_Count'] >= 1) &
    (df['Scheduled_Date'] <= df['Actual_Start_Date']) &
    (df['Actual_Start_Date'] <= df['Completion_Date'])
]

# Final duplicate safety check

In [113]:
df.duplicated().sum()

# Save Dataset

In [114]:
# Save the final cleaned dataset to CSV and Excel
df.to_csv("aircraft_maintenance_cleaned.csv", index=False)
df.to_excel("aircraft_maintenance_cleaned.xlsx", index=False)